### Компиляция и линковка.

<br />

##### Hello world одним файлом: компиляция и линковка

Напишем программу `hello_world.cpp` одним файлом:

```c++
#include <cstdio>

void print_hello_world()
{
    std::puts("hello world!");
}

int main()
{
    print_hello_world();
    return 0;
}
```

Сборка С++ - программ делится на 2 этапа:
* компиляция объектных файлов (.o/.obj) с ассемблером из исходных текстов (.cpp)
* линковка (связывание) нескольких объектных файлов в один исполняемый

Соберём компилятором clang:

In [1]:
# компилируем объектный файл:
!clang++ -Os -c hello_world.cpp -o hello_world.o

# собираем все объектные файлы (1 штука) в исполняемый:
!clang++ hello_world.o -o hello_world.exe

# проверим:
!./hello_world.exe

hello world!


Вроде работает

<br />

Проанализиурем содержимое объектного файла `hello_world.o`.

Посмотрим на символы внутри него:

In [2]:
!objdump -C -t hello_world.o


hello_world.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 hello_world.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	000000000000000a print_hello_world()
000000000000000a g     F .text	000000000000000f main
0000000000000000         *UND*	0000000000000000 puts




**Уровень видимости символа:**
* `l` - локальный символ (видимый только внутренним сущностям этого объектного файла)
* `g` - глобальный символ (видимый для других объектных файлов)

In [3]:
# наша строка:
!objdump -s --section=.rodata.str1.1 hello_world.o


hello_world.o:     file format elf64-x86-64

Contents of section .rodata.str1.1:
 0000 68656c6c 6f20776f 726c6421 00        hello world!.   


In [4]:
# код функций:
!objdump -C -d hello_world.o


hello_world.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <print_hello_world()>:
   0:	bf 00 00 00 00       	mov    $0x0,%edi
   5:	e9 00 00 00 00       	jmpq   a <main>

000000000000000a <main>:
   a:	50                   	push   %rax
   b:	bf 00 00 00 00       	mov    $0x0,%edi
  10:	e8 00 00 00 00       	callq  15 <main+0xb>
  15:	31 c0                	xor    %eax,%eax
  17:	59                   	pop    %rcx
  18:	c3                   	retq   


<br />

##### Hello world одним файлом: изменение уровня видимости символа

Напомню содержимое файла hello_world.cpp:

```c++
#include <cstdio>

void print_hello_world()
{
    std::puts("hello world!");
}

int main()
{
    print_hello_world();
    return 0;
}
```

Если внимательно присмотреться к программе:
* Функция `main` должна быть видима снаружи объектного файла (почему?). `main` - глобальный символ, это нормально.
* Функция `print_hello_world` снаружи объектного файла не используется. Этот символ можно (нужно) сделать локальным.

**Замечание:**
В терминах стандарта это называется `internal linkage` / `external linkage` (https://en.cppreference.com/w/cpp/language/storage_duration)

В С++ есть два способа сделать функцию / глобальную переменную / глобальную константу локальным символом:
* обернуть в анонимный namespace:

```c++
namespace {
    void print_hello_world()
    {
        std::puts("hello world!");
    }
}
```

* добавить ключевое слово `static`:

```c++
static void print_hello_world()
{
    std::puts("hello world!");
}
```

Сделаем символ `print_hello_world()` локальным в файле `hello_world_local.cpp` и скомпилируем:

```c++
#include <cstdio>

static void print_hello_world()
{
    std::puts("hello world!");
}

int main()
{
    print_hello_world();
    return 0;
}
```

In [5]:
# компилируем объектный файл:
!clang++ -Os -c hello_world_local.cpp -o hello_world_local.o

# собираем все объектные файлы (1 штука) в исполняемый:
!clang++ hello_world_local.o -o hello_world_local.exe

!./hello_world_local.exe

hello world!


Посмотрим на содержимое объектного файла `hello_world_local.o`:

In [6]:
!objdump -C -t hello_world_local.o


hello_world_local.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 hello_world_local.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	000000000000000f main
0000000000000000         *UND*	0000000000000000 puts




**Вопрос:** куда делась функция `print_hello_world()`?

<details>
<summary>ответ</summary>
<p>
    её код заинлайнен в `main` как очень короткий, а т.к. сама функция нигде не используется, она удалена
</p>
</details>

Проверим нашу гипотезу, посмотрим на команды в объектном файле:

In [7]:
# код функций:
!objdump -C -d hello_world_local.o


hello_world_local.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <main>:
   0:	50                   	push   %rax
   1:	bf 00 00 00 00       	mov    $0x0,%edi
   6:	e8 00 00 00 00       	callq  b <main+0xb>
   b:	31 c0                	xor    %eax,%eax
   d:	59                   	pop    %rcx
   e:	c3                   	retq   


<br />

**Рекомендация:**
* Если символ не используется за пределами одного `.cpp` - файла, его желательно сделать локальным. Локальность символа - дополнительное знание для компилятора, позволяет проводить более агрессивные оптимизации.
* Локальные символы в заголовочных файлах светить **не нужно**

<br />

##### Hello world в два файла: компиляция и линковка

Разобьём `hello_world.cpp` на 2 `.cpp` файла. Отдельно для функции `main`, отдельно для функции `print_hello_world()`:

`hello_world_2_print.h:`

```c++
#pragma once

void print_hello_world();
```

<br />

`hello_world_2_print.cpp:`

```c++
#include "hello_world_2_print.h"

#include <cstdio>

void print_hello_world()
{
    std::puts("hello world!");
}
```

<br />

`hello_world_2_main.cpp:`

```c++
#include "hello_world_2_print.h"

int main()
{
    print_hello_world();
    return 0;
}
```

In [8]:
# компилируем объектные файлы:
!clang++ -c -Os hello_world_2_print.cpp -o hello_world_2_print.o
!clang++ -c -Os hello_world_2_main.cpp  -o hello_world_2_main.o

# линкуем их в исполняемый файл:
!clang++ -Os hello_world_2_print.o hello_world_2_main.o -o hello_world_2.exe

# проверим что работает:
!./hello_world_2.exe

hello world!


<br />

Проверим содержимое объектных файлов:

In [9]:
!objdump -C -t hello_world_2_print.o


hello_world_2_print.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 hello_world_2_print.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	000000000000000a print_hello_world()
0000000000000000         *UND*	0000000000000000 puts




In [10]:
!objdump -C -t hello_world_2_main.o


hello_world_2_main.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 hello_world_2_main.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000         *UND*	0000000000000000 print_hello_world()
0000000000000000 g     F .text	000000000000000a main




*Объяснить что произошло*

<br />

**Вопрос:** почему больше не получится сделать символ `print_hello_world` локальным?

In [11]:
# почистим за собой
!rm -f *.o *.exe

<br />

##### Этапы сборки программ

https://en.cppreference.com/w/cpp/language/translation_phases

Упрощённая схема:

![](program_stages_2.jpg)

Вопросы:
* где в этом процессе header-файлы?
* как собирать не исполняемый файл, а статическую билиотеку?
* как в этой схеме обрабатываются шаблоны?
* на каких уровнях возможна параллелизация процесса?

<br />

##### Препроцессор - зачем и как

Заголовочные файлы

```c++
#include <string>

#include "utils.h"
```

Макросы автоподстановки - постарайтесь по возможности избегать их, предпочитать С++ - аналоги

```c++
#define SQR(x)  x * x


int y = SQR(5);  // int y = 5 * 5;
```

<details>
<summary>Объяснение ошибки</summary>
<p>

```c++
// error:
//     SQR(2 + 3)  --> 2 + 3 * 2 + 3 == 11  // OOOOOPS!
//
// corrected version:
#define SQR(x) (x)*(x)
```

</p>
</details>

<br />
Другой пример:

```c++
#define LOG(msg) \
    if (is_logging_enabled) \
        std::clog << (msg) << std::endl;
```

<details>
<summary>Объяснение ошибки</summary>
<p>

```c++
// error: 
//     if (something_bad_happened)
//         LOG("something bad happened!");
//     else
//         run_task();  // OOOOPS!
//
// corrected version (linux kernel programming guides):
#define LOG(msg) \
    do { \
        if (is_logging_enabled) \
            std::clog << (msg) << std::endl; \
    } while(0)
```

</p>
</details>

Генерация ошибок и предупреждений в compile-time

```c++
#error This file must not be included into compilation!
```

```c++
void f()
{
    #warning Function f() must be fixed for case when there is no network connection
    ...
}
```

Условная компиляция

```c++
// условная компиляция под разные ОС
void remove_file(const std::string& filename)
{
    #if defined(__LINUX__)
        ...
    #elif defined(__WINDOWS__)
        ...
    #elif defined(__MACOSX__)
        ...
    #elif defined(__ANDROID__)
        ...
    #elif defined(__IOS__)
        ...
    #else
        #error Unsupported target OS
    #endif
}
```

https://en.cppreference.com/w/cpp/preprocessor/replace

https://en.cppreference.com/w/cpp/preprocessor/impl

А также:
* информация о поддерживаемых фичах компилятором (`__cpp_generic_lambdas`, `__cpp_range_based_for` и др.)
* дополнительная информация для точной отладки/трассировки/профилировки (`__LINE__`, `__FILE__` и нестандартный `__function__`)
* `#pragma` - команды либо подсказки компилятору (`#pragma once`, `#pragma unroll` ...)

<br />

##### Компиляция. Содержимое объектных файлов. Область видимости символов.

Набросаем программку из нескольких файлов и посмотрим на содержимое объектных файлов

`ex1_objfiles/main.cpp`

```c++
#include "math_utils.h"
#include "string_utils.h"

#include <cstdio>
#include <cstring>

void print_help() noexcept
{
    std::puts("USAGE:");
    std::puts("    exmaple.bin mmul 3 4");
    std::puts("        12");
    std::puts("    exmaple.bin smul 3 4");
    std::puts("        3333");
    std::puts("");
    std::puts("Errors processing is not implemented in order to simplify example");
}

int main(int argc, char** argv)
{
    if (argc != 4)
    {
        print_help();
        return 1;
    }

    const char* const op = argv[1];
    const char* const a1 = argv[2];
    const char* const a2 = argv[3];
    if (!std::strcmp(op, "mmul"))
        math_utils::mul(a1, a2);
    else if (!std::strcmp(op, "smul"))
        string_utils::mul(a1, a2);
    return 0;
}
```

`ex1_objfiles/math_utils.h:`

```c++
#pragma once

namespace math_utils
{
    void mul(const char* a1, const char* a2) noexcept;
}  // namespace math_utils
```

`ex1_objfiles/string_utils.h:`

```c++
#pragma once

namespace string_utils
{
    void mul(const char* a1, const char* a2) noexcept;
}  // namespace string_utils
```

`ex1_objfiles/math_utils.cpp:`

```c++
#include "math_utils.h"

#include <cstdio>
#include <cstdlib>

int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace math_utils
{
    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils
```

`ex1_objfiles/string_utils.cpp:`

```c++
#include "string_utils.h"

#include <cstdio>
#include <cstdlib>

int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace string_utils
{
    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x2 = convert_to_int(a2);
        for (int i = 0; i < x2; ++i)
            std::puts(a1);
    }
}  // namespace string_utils
```

Скомпилируем объектные файлы:

In [1]:
!clang++ -c -Os ex1_objfiles/main.cpp
!clang++ -c -Os ex1_objfiles/math_utils.cpp
!clang++ -c -Os ex1_objfiles/string_utils.cpp

In [2]:
!ls *.o -lh

-rw-rw-r-- 1 ivafanas ivafanas 2,1K фев 10 22:16 main.o
-rw-rw-r-- 1 ivafanas ivafanas 1,4K фев 10 22:16 math_utils.o
-rw-rw-r-- 1 ivafanas ivafanas 1,3K фев 10 22:16 string_utils.o


Посмотрим на символы в main.o:

In [3]:
!objdump -C -t main.o


main.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 main.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000048 print_help()
0000000000000000         *UND*	0000000000000000 math_utils::mul(char const*, char const*)
0000000000000000         *UND*	0000000000000000 string_utils::mul(char const*, char const*)
0000000000000048 g     F .text	0000000000000066 main
0000000000000000         *UND*	0000000000000000 putchar
0000000000000000         *UND*	0000000000000000 puts
0000000000000000         *UND*	0000000000000000 strcmp




Таблица символов показывает, что:
* в этом файле определены и видны наружу `print_help` и `main`
* файл ссылается на символы `math_utils::mul`, `string_utils::mul`, `putchar`, `puts`, `strcmp`, которые реализованы где-то извне

А вот, кстати, и наши строчки:

In [4]:
!objdump -s --section=.rodata.str1.1 main.o


main.o:     file format elf64-x86-64

Contents of section .rodata.str1.1:
 0000 55534147 453a0020 20202065 786d6170  USAGE:.    exmap
 0010 6c652e62 696e206d 6d756c20 33203400  le.bin mmul 3 4.
 0020 20202020 20202020 31320020 20202065          12.    e
 0030 786d6170 6c652e62 696e2073 6d756c20  xmaple.bin smul 
 0040 33203400 20202020 20202020 33333333  3 4.        3333
 0050 00457272 6f727320 70726f63 65737369  .Errors processi
 0060 6e672069 73206e6f 7420696d 706c656d  ng is not implem
 0070 656e7465 6420696e 206f7264 65722074  ented in order t
 0080 6f207369 6d706c69 66792065 78616d70  o simplify examp
 0090 6c65006d 6d756c00 736d756c 00        le.mmul.smul.   


Посмотрим на символы `math_utils`:

In [5]:
!objdump -C -t math_utils.o


math_utils.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 math_utils.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000005 convert_to_int(char const*)
0000000000000005 g     F .text	000000000000002e math_utils::mul(char const*, char const*)
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 printf




Посмотрим на символы `string_utils`:

In [6]:
!objdump -C -t string_utils.o


string_utils.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 string_utils.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 g     F .text	0000000000000005 convert_to_int(char const*)
0000000000000005 g     F .text	0000000000000027 string_utils::mul(char const*, char const*)
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 puts




__Вопросы__:
* В программе ошибка. Возможно, линковщик её найдёт. Что за ошибка? Как починить?

ODR

In [7]:
!clang++ main.o math_utils.o string_utils.o -o example.exe

string_utils.o: In function `convert_to_int(char const*)':
string_utils.cpp:(.text+0x0): multiple definition of `convert_to_int(char const*)'
math_utils.o:math_utils.cpp:(.text+0x0): first defined here
clang: error: linker command failed with exit code 1 (use -v to see invocation)


In [8]:
# удалим за собой
!rm -f *.o *.exe *.bin *.out

<br />

Теперь подправим math_utils.cpp, скажем компилятору, что функция `covnert_to_int` снаружи никому не нужна:

`ex1_objfiles/math_utils_w_static.cpp:`

```c++
#include "math_utils.h"

#include <cstdio>
#include <cstdlib>

static int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}

namespace math_utils
{
    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils
```

In [9]:
!clang++ -Os -c ex1_objfiles/math_utils_w_static.cpp

In [10]:
!objdump -C -t math_utils_w_static.o


math_utils_w_static.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 math_utils_w_static.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	000000000000002e math_utils::mul(char const*, char const*)
0000000000000000         *UND*	0000000000000000 atoi
0000000000000000         *UND*	0000000000000000 printf




В данном случае компилятор не просто перенёс символ `convert_to_int` в локальные (можно это продемонстрировать, убрав оптимизации), но и принял решение полностью удалить функцию, заинлайнив её.

Аналогичным образом желательно спрятать функцию `convert_to_int` в `string_utils.cpp` и `print_help` в `main.cpp`.

__Вопрос__: Что делать, если хотим избавиться от копипасты `convert_to_int`? В отдельный `cpp`? В хедер? Как себя поведёт компилятор?

In [11]:
# удалим за собой
!rm -f *.o *.exe *.bin *.out

<br />

##### static vs inline vs compiler inline

Попробуем избавиться от copy-paste с `convert_to_int` разными вариантами.

__Вариант 1__: вынесем `convert_to_int` в header и объявим его `static`

`static` перед функцией означает, что символ не нужно экспортировать наружу, он должен быть локальным.

`ex2_objfiles/convert.h:`

```c++
#pragma once

#include <cstdlib>

static int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}
```

`ex2_objfiles/math_utils.cpp:`

```c++
#include "math_utils.h"

#include "convert.h"

#include <cstdio>

namespace math_utils
{
    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x1 = convert_to_int(a1);
        const int x2 = convert_to_int(a2);
        std::printf("%d\n", x1 * x2);
    }
}  // namespace math_utils
```

`ex2_objfiles/string_utils.cpp:`

```c++
#include "string_utils.h"

#include "convert.h"

#include <cstdio>

namespace string_utils
{
    void mul(const char* const a1, const char* const a2) noexcept
    {
        const int x2 = convert_to_int(a2);
        for (int i = 0; i < x2; ++i)
            std::puts(a1);
    }
}  // namespace string_utils
```

Скомпилируем, обратим особое внимание на символ `convert_to_int`

In [12]:
!clang++ -O0 -c ex2_objfiles/main.cpp
!clang++ -O0 -c ex2_objfiles/math_utils.cpp
!clang++ -O0 -c ex2_objfiles/string_utils.cpp
!clang++ -O0 main.o math_utils.o string_utils.o -o a.out

In [13]:
!objdump -C -t math_utils.o | grep convert_to_int

0000000000000060 l     F .text	000000000000001b convert_to_int(char const*)


In [14]:
!objdump -C -t string_utils.o | grep convert_to_int

0000000000000070 l     F .text	000000000000001b convert_to_int(char const*)


In [15]:
!objdump -C -t a.out | grep convert_to_int

0000000000400900 l     F .text	000000000000001b              convert_to_int(char const*)
0000000000400990 l     F .text	000000000000001b              convert_to_int(char const*)


То же самое с оптимизациями:

In [16]:
!clang++ -Os -c ex2_objfiles/main.cpp
!clang++ -Os -c ex2_objfiles/math_utils.cpp
!clang++ -Os -c ex2_objfiles/string_utils.cpp
!clang++ -Os main.o math_utils.o string_utils.o -o a.out
!objdump -t a.out | grep convert_to_int

__Итого__: 
* в случае без оптимизаций происходит то, что и должно быть: каждый объектный файл содержит свою собственную реализацию `convert_to_int`, и **функция в исполняемом файле встречается столько раз, сколько объектных файлов слинковано**
* в случае оптимизаций компилятор здесь принимает решение выкинуть функцию, заинлайнив её код. Но такое поведение зависит от размера функции и работает не всегда.

In [17]:
# удалим за собой
!rm -f *.o *.exe *.bin *.out

<br />

__Вариант 2:__ вынесем `convert_to_int` в header и объявим его `inline`

`inline` перед функцией имеет два значения:
* позволяет нарушать ODR
* рекомендация компилятору заинлайнить код функции (показать как это работает в godbolt на clang)

Отступление: показать сообщения clang об оптимизациях на примере (потом добавить std::puts в sqr2, потом убрать static-и):

```c++
static double sqr1(double x) {
    return x * x;
}

static inline double sqr2(double x) {
    return x * x;
}

double f(double x, double y) {
    return sqr1(x) + sqr2(y);
}
```

Вернёмся к `inline` и ODR

`ex3_objfiles/convert.h:`
    
```c++
#pragma once

#include <cstdlib>

inline int convert_to_int(const char* const s) noexcept
{
    return std::atoi(s);
}
```

Скомпилируем программу без оптимизаций:

In [18]:
!clang++ -O0 -c ex3_objfiles/main.cpp
!clang++ -O0 -c ex3_objfiles/math_utils.cpp
!clang++ -O0 -c ex3_objfiles/string_utils.cpp
!clang++ main.o math_utils.o string_utils.o

In [19]:
!objdump -C -t math_utils.o | grep convert_to_int

0000000000000000 l    d  .text._Z14convert_to_intPKc	0000000000000000 .text._Z14convert_to_intPKc
0000000000000000  w    F .text._Z14convert_to_intPKc	000000000000001b convert_to_int(char const*)


In [20]:
!objdump -C -t string_utils.o | grep convert_to_int

0000000000000000 l    d  .text._Z14convert_to_intPKc	0000000000000000 .text._Z14convert_to_intPKc
0000000000000000  w    F .text._Z14convert_to_intPKc	000000000000001b convert_to_int(char const*)


In [21]:
!objdump -C -t a.out | grep convert_to_int

0000000000400900  w    F .text	000000000000001b              convert_to_int(char const*)


То же самое с опитизациями:

In [22]:
!clang++ -Os -c ex3_objfiles/main.cpp
!clang++ -Os -c ex3_objfiles/math_utils.cpp
!clang++ -Os -c ex3_objfiles/string_utils.cpp
!clang++ main.o math_utils.o string_utils.o

In [23]:
!objdump -C -t math_utils.o | grep convert_to_int

In [24]:
!objdump -C -t a.out | grep convert_to_int

__Итого:__
* в случае без оптимизаций происходит то, что и должно быть: каждый объектный файл может содержать свою собственную реализацию `convert_to_int`, а функция в исполняемом файле встречается один раз.
* в случае оптимизаций компилятор здесь принимает решение выкинуть функцию, заинлайнив её код. Но такое поведение зависит от размера функции и работает не всегда.

In [25]:
# удалим за собой
!rm -f *.o *.exe *.bin *.out

<br />

##### Классы и анонимный namespace. Местонахождение методов класса.

Пару слов как компилятор работает с символами на примере классов

__Вариант 1__: класс с методами, определёнными в рамках класса

`ex4_class/example_class_1.cpp:`
    
```c++
#include <cstdio>

class HTMLPrinter {
public:
    void p() {
        print("<p>");
    }

    void div() {
        print("<div>");
    }

private:
    void print(const char* s) {
        std::puts(s);
    }
};

void f() {
    HTMLPrinter printer;
    printer.p();
}
```

In [37]:
!clang++ -O0 -c ex4_class/example_class_1.cpp
!objdump -C -t example_class_1.o


example_class_1.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_1.cpp
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .text._ZN11HTMLPrinter1pEv	0000000000000000 .text._ZN11HTMLPrinter1pEv
0000000000000000 l    d  .text._ZN11HTMLPrinter5printEPKc	0000000000000000 .text._ZN11HTMLPrinter5printEPKc
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000 g     F .text	0000000000000017 f()
0000000000000000  w    F .text._ZN11HTMLPrinter1pEv	0000000000000025 HTMLPrinter::p()
0000000000000000  w    F .text._ZN11HTMLPrinter5printEPKc	0000000000000022 HTMLPrinter::print(char const*)
0000000000000000         *UND*	0000000000000000 puts




`ex4_class/example_class_2.cpp:`

```c++
#include <cstdio>

class HTMLPrinter {
public:
    void p();
    void div();

private:
    void print(const char* s);
};

void HTMLPrinter::p() {
    print("<p>");
}

void HTMLPrinter::div() {
    print("<div>");
}

void HTMLPrinter::print(const char* const s) {
    std::puts(s);
}

void f() {
    HTMLPrinter printer;
    printer.p();
}
```

In [38]:
!clang++ -O0 -c ex4_class/example_class_2.cpp
!objdump -C -t example_class_2.o


example_class_2.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_2.cpp
0000000000000000 l       .gcc_except_table	0000000000000000 GCC_except_table5
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .gcc_except_table	0000000000000000 .gcc_except_table
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000         *UND*	0000000000000000 _Unwind_Resume
00000000000000b0 g     F .text	0000000000000050 f()
0000000000000020 g     F .text	0000000000000025 HTMLPrinter::p()
0000000000000080 g     F .text	0000000000000025 HTMLPrinter::div()
0000000000000050 g     F .text	0000000000000022 HTMLPrinter::print(char const*)
0000000000000000 g     F .text	000000000000000a HTMLPrinter::HTMLPrinter()
0000000000000000 g     F .text	000000000000000a HTMLPrinter::HTMLPrinter()
0000000000000010 g     F .text	000000000000000a HTMLPrinter::~HTMLPrinter()
0000000000000010 g     F .text	00

**Способ организации класса 1:** реализация методов сразу в header

```c++
// HTMLPrinter.h
class HTMLPrinter {
 public:
    HTMLPrinter() {}
    void p() { ...; }
    
 private:
    void print() { ...; }
};
```

Особенности:
* (-) долгая компиляция
* (-) объектные файлы больше весят
* (+) лучше оптимизации

**Способ организации класса 2:** разделение на header && cpp, вынос методов в cpp

```c++
// HTMLPrinter.h
class HTMLPrinter {
 public:
    HTMLPrinter();
    void p();
    
 private:
    void print();
};

// HTMLPrinter.cpp
HTMLPrinter::HTMLPrinter() = default;
void HTMLPrinter::p() { ...; }
void HTMLPrinter::print() { ...; }
```

Особенности:

* (+) быстрая компиляция
* (+) объектные файлы меньше весят
* (-) хуже оптимизации

*Объяснить разницу*

<br />

**Способ сделать класс с `internal linkage`:**

`ex4_class/example_class_3.cpp:`

```c++
#include <cstdio>

namespace {
  class HTMLPrinter {
  public:
      HTMLPrinter();
      ~HTMLPrinter();

      void p();
      void div();

  private:
      void print(const char* s);
  };

  HTMLPrinter::HTMLPrinter() = default;
  HTMLPrinter::~HTMLPrinter() = default;

  void HTMLPrinter::p() {
      print("<p>");
  }

  void HTMLPrinter::div() {
      print("<div>");
  }

  void HTMLPrinter::print(const char* const s) {
      std::puts(s);
  }
}  // namespace

void f() {
    HTMLPrinter printer;
    printer.p();
}
```

In [39]:
!clang++ -O0 -c ex4_class/example_class_3.cpp
!objdump -C -t example_class_3.o


example_class_3.o:     file format elf64-x86-64

SYMBOL TABLE:
0000000000000000 l    df *ABS*	0000000000000000 example_class_3.cpp
0000000000000000 l       .gcc_except_table	0000000000000000 GCC_except_table0
0000000000000060 l     F .text	0000000000000025 (anonymous namespace)::HTMLPrinter::p()
00000000000000a0 l     F .text	0000000000000022 (anonymous namespace)::HTMLPrinter::print(char const*)
0000000000000050 l     F .text	000000000000000a (anonymous namespace)::HTMLPrinter::HTMLPrinter()
0000000000000090 l     F .text	000000000000000a (anonymous namespace)::HTMLPrinter::~HTMLPrinter()
0000000000000000 l    d  .text	0000000000000000 .text
0000000000000000 l    d  .gcc_except_table	0000000000000000 .gcc_except_table
0000000000000000 l    d  .rodata.str1.1	0000000000000000 .rodata.str1.1
0000000000000000         *UND*	0000000000000000 _Unwind_Resume
0000000000000000 g     F .text	0000000000000050 f()
0000000000000000         *UND*	0000000000000000 __gxx_personality_v0
00000000000000

<br />

##### Шаблоны и символы

Рассмотрим пример с `std::vector`

`ex5_templates/util.h:`
    
```c++
#pragma once

#include <vector>

int sum(const std::vector<int>& v);
```

`ex5_templates/util.cpp:`

```c++
#include "util.h"

int sum(const std::vector<int>& v)
{
    auto v_copy = v;
    int rv = 0;
    for (int x : v_copy)
        rv += x;
    return rv;
}
```

`ex5_templates/main.cpp:`

```c++
#include "util.h"

#include <cstdio>
#include <vector>

int mul(const std::vector<int>& v)
{
    auto v_copy = v;
    int rv = 1;
    for (int x : v_copy)
        rv *= x;
    return rv;
}

int main(int argc, char** argv)
{
    std::vector<int> v = {1, 2, 3};
    std::printf("%d", mul(v));
    std::printf("%d", sum(v));
    return 0;
}
```

Скомпилируем, обратим внимание на `vector`

In [40]:
!clang++ -Os -c ex5_templates/main.cpp
!clang++ -Os -c ex5_templates/util.cpp
!clang++ -Os main.o util.o

In [41]:
!objdump -C -t util.o | grep -i vector

0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000000000 l    d  .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000000 .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000 g     F .text	0000000000000046 sum(std::vector<int, std::allocator<int> > const&)
0000000000000000  w    F .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000026 std::_Vector_base<int, std::allocator<int> >::_M_allocate(unsigned long)
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000079 std::vector<int, std::allocator<int> >::vector(std::vector<int, std::allocator<int> > const&)


In [42]:
!objdump -C -t main.o | grep -i vector

0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ERKS1_
0000000000000000 l    d  .text._ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_	0000000000000000 .text._ZNSt6vectorIiSaIiEEC2ESt16initializer_listIiERKS0_
0000000000000000 l    d  .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000000 .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm
0000000000000000 g     F .text	000000000000004a mul(std::vector<int, std::allocator<int> > const&)
0000000000000000         *UND*	0000000000000000 sum(std::vector<int, std::allocator<int> > const&)
0000000000000000  w    F .text._ZNSt12_Vector_baseIiSaIiEE11_M_allocateEm	0000000000000026 std::_Vector_base<int, std::allocator<int> >::_M_allocate(unsigned long)
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ERKS1_	0000000000000079 std::vector<int, std::allocator<int> >::vector(std::vector<int, std::allocator<int> > const&)
0000000000000000  w    F .text._ZNSt6vectorIiSaIiEEC2ESt1

In [43]:
!objdump -C -t a.out | grep -i vector

0000000000400912  w    F .text	0000000000000026              std::_Vector_base<int, std::allocator<int> >::_M_allocate(unsigned long)
0000000000400818  w    F .text	0000000000000079              std::vector<int, std::allocator<int> >::vector(std::vector<int, std::allocator<int> > const&)
0000000000400938 g     F .text	0000000000000046              sum(std::vector<int, std::allocator<int> > const&)
0000000000400748 g     F .text	000000000000004a              mul(std::vector<int, std::allocator<int> > const&)
0000000000400892  w    F .text	000000000000007f              std::vector<int, std::allocator<int> >::vector(std::initializer_list<int>, std::allocator<int> const&)


In [44]:
# удалим за собой
!rm -f *.o *.exe *.bin *.out

<br />

##### Extern templates

Возможность сэкономить на времени компиляции, указав компилятору, что "где-то в другом месте этот шаблон уже скомпилирован, компилировать не надо, просто сделай внешнюю ссылку".

Подробно разбирать не будем из-за редкости использования.

https://stackoverflow.com/questions/8130602/using-extern-template-c11

<br />

##### Как определить глобальную константу в хедере

Объяснить варианты и разницу:

* `static`
    ```c++
    // in header - bad
    static const std::string s = "123";
    
    // in single cpp - good
    static const std::string s = "123";
    ```
* `inline`
    ```c++
    // in header - +-
    inline const std::string s = "123";

    // in single cpp - bad
    inline const std::string s = "123";
    ```
* `extern` - good
    ```c++
    // in header
    extern const std::string s;
    
    // in a single cpp
    const std::string s = "123";
    ```

<br />

##### Линковка

Задача линковщика - собрать объектные файлы (.o/.obj) в исполняемый файл.

Линковщик должен:
1. Разрешить зависимости и построить связи
2. Смёржить символы, которым разрешено нарушать ODR

In [45]:
# compilation
!clang++ -Os -c ex5_templates/main.cpp
!clang++ -Os -c ex5_templates/util.cpp

# linking
!clang++ -Os main.o util.o -o a.out

# cleanup
!rm -f *.o a.out

https://docs.microsoft.com/en-us/cpp/build/reference/linking?view=vs-2019

https://gcc.gnu.org/onlinedocs/gcc/Link-Options.html

<br />

##### Загрузка и запуск программы

https://en.cppreference.com/w/cpp/language/main_function

Что происходит, когда вы вызываете из командной строки `my_program.exe`? Или когда щёлкаете мышкой по exe-файлу в проводнике?

Простой вариант:
1. Загрузка кода программы с диска в ОП (возможны варианты)
2. Инициализация глобальных переменных и констант
    * в рамках одной единицы трансляции порядок гарантирован
    * между единицами трансляции порядок не определён
3. Выполнение `main`

Здесь баг:

`f1.h:`

```c++
#pragma once
extern const std::string s1;
```

`f1.cpp:`

```c++
#include "f1.h"
const std::string s1 = "123";
```
        
`f2.h:`

```c++
#pragma once
extern const std::string s2;
```
        
`f2.cpp:`

```c++
#include "f2.h"
#include "f1.h"
const std::string s2 = s1 + "456";
```

`main.cpp:`

```c++
#include "f2.h"

int main() {
    std::cout << s2 << std::endl;  // ???
    return 0;
}
```

__Вопрос__: Почему? Как это можно починить?


<details>
<summary>Вариант:</summary>
<p>

`f.h:`

```c++
#pragma once
extern const std::string s1;
extern const std::string s2;
```
        
`f.cpp:`

```c++
#include "f.h"
const std::string s1 = "123";
const std::string s2 = s1 + "456";
```

`main.cpp:`

```c++
#include "f.h"

int main() {
    std::cout << s2 << std::endl;
    return 0;
}
```

</p>
</details>

<br />

##### Запуск программы и shared libs

Код может линковаться не только со статическими билиотеками, но и с динамическими. Динамическая билиотека - набор функций, которые не вкопилируются в исполняемый файл, а хранятся отдельным файлом. В исполняемом файле прописывается специальная информация для функций, лежащих в динамических билиотеках.

Плюсы:
* Можно сэкономить место на диске и использовать одну динамическую библиотеку (например, libc) для нескольких программ
* Аналогичным образом экономится место в ОП под код
* Можно внести исправления в динамическую библиотеку, перекомпилировать саму программу при этом не нужно

Минусы:
* Нужно размечать экспортируемые функции вручную
* Невозможен ряд оптимизаций (inlining(!!!), LTO, constant propagation etc ...)
* Невозможен анализ компилятором неиспользуемого кода в динамической библиотеке (если функция помечена на экспорт, выкидывать её нельзя, и она тянет за собой всё по графу зависимостей)
* Нужно загружать динамическую библиотеку в память:
    * либо лениво при первом вызове функции
    * либо сразу при старте приложения (настраиваемо)
* Нужно проводить доп. работу по инициализации адресов функций
* Дороже вызов импортированной функции (особенно первый)